# Duplicate Check and Delete
This notebook is responsible for maintaining data quality by identifying and removing duplicate entries from the Google Sheets.

## Duplicate Removal Logic
The code below:
1. Connects to the 'RealEstateListings' spreadsheet.
2. Iterates through all defined worksheets (kwsintmaarten, sunshine, trust, etc.).
3. Identifies duplicates based on the **Link** column.
4. Safely removes duplicate rows while preserving empty rows or formatting where possible.
5. Utilizes batch processing to respect API rate limits.

In [ ]:
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread.exceptions import APIError

# ----------------------------------
# Google Sheets Setup
# ----------------------------------
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("secret.json", scope)
client = gspread.authorize(creds)

SPREADSHEET_NAME = "RealEstateListings"
spreadsheet = client.open(SPREADSHEET_NAME)

SHEETS = [
    "all_listings",
    "kwsintmaarten", "sunshine", "trust",
    "easyx", "century", "cornerstone", "ireteam"
]

# ----------------------------------
# Duplicate Remover (Skip Empty Rows)
# ----------------------------------
def remove_duplicates_safely(sheet, batch_size=50, wait_time=60):
    print(f"\n🧹 Cleaning duplicates in '{sheet.title}'...")
    all_rows = sheet.get_all_values()

    if not all_rows or len(all_rows) < 2:
        print("⚠️ Not enough data.")
        return

    header = all_rows[0]
    data = all_rows[1:]

    try:
        link_index = header.index("Link")
    except ValueError:
        print("❌ 'Link' column not found.")
        return

    seen_links = set()
    duplicates = []

    for i, row in enumerate(data, start=2):  # Row 2 onward
        if len(row) <= link_index:
            continue

        link = row[link_index].strip()

        # 🚫 Skip empty or whitespace-only links
        if not link or link == "":
            continue

        # ✅ Track duplicates
        if link in seen_links:
            duplicates.append(i)
        else:
            seen_links.add(link)

    total = len(duplicates)
    if total == 0:
        print("✨ No duplicates found.")
        return

    print(f"🗑️ Found {total} duplicates to remove.")
    duplicates.sort(reverse=True)  # Delete from bottom up

    deleted = 0
    for start in range(0, total, batch_size):
        chunk = duplicates[start:start + batch_size]
        print(f"📦 Batch {start//batch_size + 1}: Deleting {len(chunk)} rows...")

        for row_num in chunk:
            try:
                # Recheck current row count
                current_count = len(sheet.get_all_values())
                if row_num > current_count:
                    continue

                # ✅ Delete the row only if it still exists
                sheet.delete_rows(row_num)
                deleted += 1
                time.sleep(0.5)
            except APIError as e:
                print(f"⚠️ Skipping row {row_num} due to APIError: {e}")
                time.sleep(3)

        if start + batch_size < total:
            print(f"⏳ Waiting {wait_time}s before next batch...")
            time.sleep(wait_time)

    print(f"✅ Done — deleted {deleted} duplicates from '{sheet.title}'.")

# ----------------------------------
# Run for Each Sheet
# ----------------------------------
for name in SHEETS:
    try:
        ws = spreadsheet.worksheet(name)
        remove_duplicates_safely(ws)
    except gspread.exceptions.WorksheetNotFound:
        print(f"⚠️ Sheet '{name}' not found — skipping.")

print("\n🏁 All done! Only duplicate links removed — empty rows preserved.")
